# 0. Imports

In [84]:
from bs4 import BeautifulSoup

import requests

import pandas as pd
import numpy as np

from time import sleep

from selenium import webdriver 
from webdriver_manager.chrome import ChromeDriverManager  
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException 

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-geopy-app")
import random
import re
import datetime
import json
import math

# import suppor functions
import sys 
sys.path.append("..")
# from src.support_scraping import scrape

# 1. Scraping the different informations

For this project, our clients requested that we organise and plan their holiday, including flight, accommodation and activities suggestions according to the following restrictions:
- Holiday dates must be from Friday 8th to Sunday 17th November. 
- Flights should be direct from Madrid to Paris, on the 8th November, to Berlin on the 13th and back to Madrid on the 17th.
- Accommodation must be under 100 euros per night for a couple
- Total activity budget is X
- Activities should be about X, Y and Z
- Activities suggested should not overlap

## Activities

Requirements are:
- Total activity budget is X
- Activities should be about X, Y and Z
- Activities suggested should not overlap

Therefore, the fields to be extracted should ideally be, at least:
- Date [X]
- Time [X]
- Address [X]
- Duration [X]
- Price [X]
- Name [X]
- Description [X]
- Category [X]
- Image 
- URL 
- Score (optional)
- Score (optional)
- Reviews (Optional)
- Language (Optional)

Option 1: Fast beautiful soup:

In [4]:
musement_link = "https://www.musement.com/uk/chiang-mai/thapae-muay-thai-boxing-stadium-tickets-168514/"
civitatis_link = "https://www.civitatis.com/es/madrid/?fromDate=2024-10-21&toDate=2024-10-27"

driver = webdriver.Chrome()

driver.get(civitatis_link)

requests = requests.get(civitatis_link)

sopa = BeautifulSoup(requests.content, "html.parser")
sopa

<!DOCTYPE html>

<html data-lang="es" lang="es">
<head>
<script type="text/javascript">window.gdprAppliesGlobally=true;(function(){function n(e){if(!window.frames[e]){if(document.body&&document.body.firstChild){var t=document.body;var r=document.createElement("iframe");r.style.display="none";r.name=e;r.title=e;t.insertBefore(r,t.firstChild)}else{setTimeout(function(){n(e)},5)}}}function e(r,a,o,s,c){function e(e,t,r,n){if(typeof r!=="function"){return}if(!window[a]){window[a]=[]}var i=false;if(c){i=c(e,n,r)}if(!i){window[a].push({command:e,version:t,callback:r,parameter:n})}}e.stub=true;e.stubVersion=2;function t(n){if(!window[r]||window[r].stub!==true){return}if(!n.data){return}var i=typeof n.data==="string";var e;try{e=i?JSON.parse(n.data):n.data}catch(t){return}if(e[o]){var a=e[o];window[r](a.command,a.version,function(e,t){var r={};r[s]={returnValue:e,success:t,callId:a.callId};n.source.postMessage(i?JSON.stringify(r):r,"*")},a.parameter)}}if(typeof window[r]!=="function"){window[r

In [46]:
script_with_info = sopa.findAll("script",{"type":"application/ld+json"})[-1]

import re 



string = script_with_info.string.replace("\n","")

activities_json_dict = json.loads(string)
activities_json_dict

{'@context': 'http://schema.org',
 '@type': 'ItemList',
 'name': 'Actividades destacadas',
 'url': 'https://www.civitatis.com/es/madrid/',
 'numberOfItems': '145',
 'itemListOrder': 'Unordered',
 'itemListElement': [{'@type': 'ListItem',
   'position': 1,
   'url': 'https://www.civitatis.com/es/madrid/visita-guiada-palacio-real/',
   'image': 'https://cdn.civitatis.com/guias/madrid/fotos/actividades/visita-guiada-palacio-real.jpg',
   'name': 'Visita guiada por el Palacio Real',
   'description': 'En esta visita guiada&amp;nbsp;&lt;b&gt;descubriremos la historia de la realeza y los secretos mejor guardados del Palacio Real de Madrid&lt;/b&gt;, uno de los más grandes de Europa.'},
  {'@type': 'ListItem',
   'position': 2,
   'url': 'https://www.civitatis.com/es/madrid/excursion-toledo-segovia/',
   'image': 'https://cdn.civitatis.com/guias/madrid/fotos/actividades/excursion-toledo-segovia.jpg',
   'name': 'Excursión a Toledo y Segovia',
   'description': 'En este tour visitaremos &lt;b&

Test if beautiful soup is enough to extract information about the activity:

In [49]:
activities = pd.DataFrame(activities_json_dict["itemListElement"])
activities["url"][0]

'https://www.civitatis.com/es/madrid/visita-guiada-palacio-real/'

In [50]:
first_activity_link = "https://www.civitatis.com/es/madrid/visita-guiada-palacio-real/"

# driver = webdriver.Chrome()

# driver.get(civitatis_link)

requests = requests.get(first_activity_link)

sopa = BeautifulSoup(requests.content, "html.parser")
sopa

<!DOCTYPE html>

<html data-lang="es" lang="es">
<head>
<script type="text/javascript">window.gdprAppliesGlobally=true;(function(){function n(e){if(!window.frames[e]){if(document.body&&document.body.firstChild){var t=document.body;var r=document.createElement("iframe");r.style.display="none";r.name=e;r.title=e;t.insertBefore(r,t.firstChild)}else{setTimeout(function(){n(e)},5)}}}function e(r,a,o,s,c){function e(e,t,r,n){if(typeof r!=="function"){return}if(!window[a]){window[a]=[]}var i=false;if(c){i=c(e,n,r)}if(!i){window[a].push({command:e,version:t,callback:r,parameter:n})}}e.stub=true;e.stubVersion=2;function t(n){if(!window[r]||window[r].stub!==true){return}if(!n.data){return}var i=typeof n.data==="string";var e;try{e=i?JSON.parse(n.data):n.data}catch(t){return}if(e[o]){var a=e[o];window[r](a.command,a.version,function(e,t){var r={};r[s]={returnValue:e,success:t,callId:a.callId};n.source.postMessage(i?JSON.stringify(r):r,"*")},a.parameter)}}if(typeof window[r]!=="function"){window[r

Getting the dates is not enough with beautiful soup. So, it is better to be sure that the dates are already correct for every item tested.

### Option 2: Selenium to pget HTMLs, beautiful soup to parse it all

Paris

In [52]:
musement_link = "https://www.musement.com/uk/chiang-mai/thapae-muay-thai-boxing-stadium-tickets-168514/"
civitatis_link_paris = "https://www.civitatis.com/es/madrid/?fromDate=2024-11-09&toDate=2024-11-12"

driver = webdriver.Chrome()

driver.get(civitatis_link_paris)

html_content = driver.page_source

# sleep(3)
# driver.execute_script('window.scrollBy(0, 2000)')

# get number of pages to iterate over

sopa = BeautifulSoup(html_content)
sopa

<html class="xlarge" data-lang="es" id="html" lang="es"><head>
<link as="script" href="/didomi/" rel="preconnect"/><link as="script" href="/didomi/" rel="dns-prefetch"/><link as="script" href="/didomi/60ac3ba7-1ae5-42dc-be9a-50140ae6cea1/loader.js?target=www.civitatis.com" rel="preload"/><script async="" src="https://www.googletagmanager.com/gtag/destination?id=AW-881088637&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-216706797&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/destination?id=G-QJQ54CTKPM&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/destination?id=DC-8239069&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" fetchpriority="high" src="/gtm/gtm.js?id=GTM-NWH47P4"></script><script async="" charset="utf-8" id="spcloader" src="/didom

Queries in this page are easy, as they follow a clear pattern to navigate by city, dates and page number:
"https://www.civitatis.com/es/{CITY_NAME}/?page={PAGE_NUMBER}&fromDate={DATE_START}&toDate={DATE_START}"

Consequently, already having the city and the date values, a simple for range loop can manage the navigation with selenium, parsing pagination html with Beautiful Soup.

In [280]:
def get_pagination_htmls_by_city_date(city_name, date_start, date_end, page_start, n_pages, driver):

    html_contents = list()

    for page_number in range(page_start, n_pages + page_start):

        activities_link = f"https://www.civitatis.com/es/{city_name}/?page={page_number}&fromDate={date_start}&toDate={date_end}"

        driver.get(activities_link)
        driver.maximize_window()
        
        driver.implicitly_wait(20)
        driver.execute_script('window.scrollBy(0, 4000)')
        driver.find_element("css selector","div.m-availability")
        

        html_content = driver.page_source

        html_contents.append(html_content)

    return html_contents

Get each page's html with selenium to be parsed later all at one with beautiful soup.

In [322]:

city_name = "paris"
date_start = "2024-11-09"
date_end = "2024-11-13"

first_link = f"https://www.civitatis.com/es/{city_name}/?fromDate={date_start}&toDate={date_end}"

driver = webdriver.Chrome()
driver.maximize_window()
driver.get(first_link)

# make sure availability cards and amount of available activities show in the page
driver.implicitly_wait(20)
driver.find_element("css selector","div.m-availability")
driver.find_element("css selector","#activitiesShowing")

html_content1 = driver.page_source

soup = BeautifulSoup(html_content1)

last_page = math.ceil(int(soup.find("div",{"class","columns o-pagination__showing"}).find("div",{"class":"left"}).text.split()[0])/20)
last_page

5

In [323]:

page_start = 2
n_pages = last_page - 1

html_contents = get_pagination_htmls_by_city_date(city_name, date_start, date_end, page_start, n_pages, driver)

html_contents.append(html_content1)

# scrape all contents from each html with beautiful soup


In [324]:
first_soup = BeautifulSoup(html_contents[0])
first_activity = first_soup.findAll("div",{"class","o-search-list__item"})[0]
first_activity

<div class="o-search-list__item" data-activitiesitem="531" id="activitiesItem_531"><article class="comfort-card _cancelation __with-dates" data-activity="531" data-categories="" data-countryid="16" data-latitude="0" data-longitude="0"> <a class="comfort-card__like js-lightbox is-js-enabled" data-comebackbutton="true" data-comebackbuttontext="Mi cuenta" data-extra-class="--login-general _comeback-wrapper login-form-new" data-fav-info='{"cityId": "16", "productId": "531", "productType": "1"}' data-fav-info-city="París" data-gtm-new-model-add-to-wishlist="" data-target-id="generic-login-favs-form" href="#" id="activity-like-button-0" title="like"> <span class="comfort-card__like__label">like</span> </a> <a class="ga-trackEvent-element _activity-link" data-eventaction="click" data-eventcategory="Actividades Listado" data-eventlabel="Tour privado por París" data-gtm-activity="531" data-gtm-new-model-click='{"event":"productClick","ecommerce":{"currencyCode":"EUR","click":{"actionField":{"li

In [325]:
# find availability, available elements. TO IMPROVE
availability_elements = list(set(first_activity.findAll("div", {"class":"m-availability__item"})) - set(first_activity.findAll("div", {"class":"m-availability__item _no-dates"})))
availability_elements 


[<div class="m-availability__item">l<br/>11<div class="tooltip top m-calendar-tooltip _availability _civ-tooltip _length_4"><div class="m-availability__tooltip _4"><span class="_time">9:00</span><span class="_time">10:00</span><span class="_time">14:00</span><span class="_time">16:00</span></div></div></div>,
 <div class="m-availability__item">s<br/>09<div class="tooltip top m-calendar-tooltip _availability _civ-tooltip _length_4"><div class="m-availability__tooltip _4"><span class="_time">9:00</span><span class="_time">10:00</span><span class="_time">14:00</span><span class="_time">16:00</span></div></div></div>,
 <div class="m-availability__item">d<br/>10<div class="tooltip top m-calendar-tooltip _availability _civ-tooltip _length_4"><div class="m-availability__tooltip _4"><span class="_time">9:00</span><span class="_time">10:00</span><span class="_time">14:00</span><span class="_time">16:00</span></div></div></div>,
 <div class="m-availability__item">m<br/>12<div class="tooltip top 

In [326]:
availability_elements

[<div class="m-availability__item">l<br/>11<div class="tooltip top m-calendar-tooltip _availability _civ-tooltip _length_4"><div class="m-availability__tooltip _4"><span class="_time">9:00</span><span class="_time">10:00</span><span class="_time">14:00</span><span class="_time">16:00</span></div></div></div>,
 <div class="m-availability__item">s<br/>09<div class="tooltip top m-calendar-tooltip _availability _civ-tooltip _length_4"><div class="m-availability__tooltip _4"><span class="_time">9:00</span><span class="_time">10:00</span><span class="_time">14:00</span><span class="_time">16:00</span></div></div></div>,
 <div class="m-availability__item">d<br/>10<div class="tooltip top m-calendar-tooltip _availability _civ-tooltip _length_4"><div class="m-availability__tooltip _4"><span class="_time">9:00</span><span class="_time">10:00</span><span class="_time">14:00</span><span class="_time">16:00</span></div></div></div>,
 <div class="m-availability__item">m<br/>12<div class="tooltip top 

In [327]:
# find available days. NOTA: ILL HAVE TO HANDLE LAST AND FIRST DAYS OF MONTH CAREFULLY
availability_elements[0].find('br').next_sibling.strip()
available_days = [element.find('br').next_sibling.strip() for element in availability_elements]
available_days

['11', '09', '10', '12', '13']

In [328]:
# for each day, find availability times
availability_elements[0].find_all("span",{"class":"_time"})
available_times = [[time.text for time in element.find_all("span",{"class":"_time"})] for element in availability_elements]
available_times
# if there are available days but no available times, say "Consult with activity service provider"

[['9:00', '10:00', '14:00', '16:00'],
 ['9:00', '10:00', '14:00', '16:00'],
 ['9:00', '10:00', '14:00', '16:00'],
 ['9:00', '10:00', '14:00', '16:00'],
 ['9:00', '10:00', '14:00', '16:00']]

In [329]:
first_activity

<div class="o-search-list__item" data-activitiesitem="531" id="activitiesItem_531"><article class="comfort-card _cancelation __with-dates" data-activity="531" data-categories="" data-countryid="16" data-latitude="0" data-longitude="0"> <a class="comfort-card__like js-lightbox is-js-enabled" data-comebackbutton="true" data-comebackbuttontext="Mi cuenta" data-extra-class="--login-general _comeback-wrapper login-form-new" data-fav-info='{"cityId": "16", "productId": "531", "productType": "1"}' data-fav-info-city="París" data-gtm-new-model-add-to-wishlist="" data-target-id="generic-login-favs-form" href="#" id="activity-like-button-0" title="like"> <span class="comfort-card__like__label">like</span> </a> <a class="ga-trackEvent-element _activity-link" data-eventaction="click" data-eventcategory="Actividades Listado" data-eventlabel="Tour privado por París" data-gtm-activity="531" data-gtm-new-model-click='{"event":"productClick","ecommerce":{"currencyCode":"EUR","click":{"actionField":{"li

In [330]:
# address: use latitude and longitude, then convert with geopy
latitude = first_activity.find("article", recursive=False)["data-latitude"]

longitude = first_activity.find("article", recursive=False)["data-longitude"]

address = geolocator.reverse((latitude, longitude)).address

address

AttributeError: 'NoneType' object has no attribute 'address'

In [331]:
activity_name = first_activity.find("a",{"class":"ga-trackEvent-element _activity-link"})["title"]
activity_name

'Tour privado por París'

In [332]:
# precio
price = json.loads(first_activity.find("a",{"class":"ga-trackEvent-element _activity-link"})["data-gtm-new-model-click"])["ecommerce"]["click"]["products"][0]["price"]

# moneda
currency = json.loads(first_activity.find("a",{"class":"ga-trackEvent-element _activity-link"})["data-gtm-new-model-click"])["ecommerce"]["currencyCode"]


In [333]:
# category 
category = first_activity.find("div",{"class":"comfort-card__features"}).findAll("span")[1].text.strip()

# duration
duration = first_activity.find("div",{"class":"comfort-card__features"}).findAll("span")[0].text.strip()

In [334]:
# description
description = first_activity.find("div",{"class":"comfort-card__text l-list-card__text"}).text.strip().replace("\xa0"," ")

In [335]:
first_activity.find("img")

<img alt="Tour privado por París" class="b-lazy b-loaded bloaded" src="/f/francia/paris/tour-privado-paris-list.jpg"/>

In [339]:
# image 
image = "www.civitatis.com" + first_activity.find("img")["src"]
image

'www.civitatis.com/f/francia/paris/tour-privado-paris-list.jpg'

In [338]:
# url
url = "www.civitatis.com" + first_activity.find("a",{"data-eventcategory":"Actividades Listado"})["href"]
url

'www.civitatis.com/es/paris/tour-privado-paris/'

In [356]:
def scrape_activities_from_page(page_soup):
    activity_data_dict = {
            "activity_name": [],
            "description": [],
            "url": [],
            "image": [],
            "image2": [],
            "available_days": [],
            "available_times": [],
            "duration": [],
            "latitude": [],
            "longitude": [],
            "address": [],
            "price": [],
            "currency": [],
            "category": []
    }
    for element in page_soup.findAll("div",{"class","o-search-list__item"}):

        availability_cards = list(set(element.findAll("div", {"class": "m-availability__item"})) - 
                                set(element.findAll("div", {"class": "m-availability__item _no-dates"})))
        

        activity_scraper_dict = {
            # activity name
            "activity_name": lambda element: element.find("a", {"class": "ga-trackEvent-element _activity-link"})["title"],
            
            # description
            "description": lambda element: element.find("div", {"class": "comfort-card__text l-list-card__text"}).text.strip().replace("\xa0", " "),

            # url 
            "url": lambda element: "www.civitatis.com" + element.find("a",{"data-eventcategory":"Actividades Listado"})["href"],

            # image/gif
            "image": lambda element: "www.civitatis.com" + element.find("img")["src"],

            # image/gif
            "image2": lambda element: "www.civitatis.com" + element.find("img")["data-src"],

            # find available days. NOTA: ILL HAVE TO HANDLE LAST AND FIRST DAYS OF MONTH CAREFULLY
            "available_days": lambda _: [el.find('br').next_sibling.strip() for el in availability_cards],
            
            # available times per day
            "available_times": lambda _: [[time.text for time in el.find_all("span", {"class": "_time"})] for el in availability_cards],
            
            # duration
            "duration": lambda element: element.find("div", {"class": "comfort-card__features"}).findAll("span")[0].text.strip(),
            
            # address: use latitude and longitude, then convert with geopy
            "latitude": lambda element: element.find("article", recursive=False)["data-latitude"],
            "longitude": lambda element: element.find("article", recursive=False)["data-longitude"],

            "address": lambda element: geolocator.reverse(
                (element.find("article", recursive=False)["data-latitude"], 
                element.find("article", recursive=False)["data-longitude"])
            ).address,
    
            # price
            "price": lambda element: json.loads(element.find("a", {"class": "ga-trackEvent-element _activity-link"})["data-gtm-new-model-click"])["ecommerce"]["click"]["products"][0]["price"],
            
            # price currency
            "currency": lambda element: json.loads(element.find("a", {"class": "ga-trackEvent-element _activity-link"})["data-gtm-new-model-click"])["ecommerce"]["currencyCode"],
            
            # category 
            "category": lambda element: element.find("div", {"class": "comfort-card__features"}).findAll("span")[1].text.strip(),
        }


        for key, activity_scraper_function in activity_scraper_dict.items():
            try:
                activity_data_dict[key].append(activity_scraper_function(element))
                
            except Exception as e:
                print(f"Error filling {key} due to {e}")
                activity_data_dict[key].append(np.nan)

    print(activity_data_dict)
    return activity_data_dict




In [358]:

# define url variables
city_name = "berlin"
date_start = "2024-11-09"
date_end = "2024-11-13"

first_link = f"https://www.civitatis.com/es/{city_name}/?fromDate={date_start}&toDate={date_end}"

# open driver
driver = webdriver.Chrome()
driver.maximize_window()
driver.get(first_link)

# make sure availability cards and amount of available activities show in the page
driver.implicitly_wait(20)
driver.find_element("css selector","div.m-availability")
driver.find_element("css selector","#activitiesShowing")

# parse first page to get last page number
html_content1 = driver.page_source

soup = BeautifulSoup(html_content1)

last_page = math.ceil(int(soup.find("div",{"class","columns o-pagination__showing"}).find("div",{"class":"left"}).text.split()[0])/20)
last_page

page_start = 2
n_pages = last_page - 1

# collect unparsed pages html
html_contents = get_pagination_htmls_by_city_date(city_name, date_start, date_end, page_start, n_pages, driver)
html_contents.append(html_content1)

# parse each page and scrape elements
total_actitivities_df = pd.DataFrame()
for page_html in html_contents:

    page_soup = BeautifulSoup(page_html)
    page_activities_df = pd.DataFrame(scrape_activities_from_page(page_soup))
    total_actitivities_df = pd.concat([total_actitivities_df,page_activities_df]).reset_index(drop=True)
    

total_actitivities_df

Error filling image2 due to 'data-src'
Error filling image2 due to 'data-src'
Error filling address due to 'NoneType' object has no attribute 'address'
Error filling address due to HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=52.5168789&lon=13.3813841&format=json&addressdetails=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000002AAE06DBB90>, 'Connection to nominatim.openstreetmap.org timed out. (connect timeout=1)'))
Error filling category due to list index out of range
Error filling category due to list index out of range
Error filling address due to 'NoneType' object has no attribute 'address'
Error filling image2 due to 'data-src'
Error filling address due to 'NoneType' object has no attribute 'address'
Error filling image2 due to 'data-src'
Error filling image2 due to 'data-src'
Error filling category due to list index out of range
Error filling image2 due to 'data-src'
Error fillin

,activity_name,description,url,image,image2,available_days,available_times,duration,latitude,longitude,address,price,currency,category
0,Entrada a la Torre de la Televisión,Podréis reservar la entrada a la Torre de la T...,www.civitatis.com/es/berlin/entrada-torre-tele...,www.civitatis.com/f/alemania/berlin/entrada-to...,NaN,[],[],30 minutos,52.520849,13.4096,"Berliner Fernsehturm, 1a, Panoramastraße, Span...",2.65,EUR,Entradas
1,Visita guiada por el Museo Nuevo,¿Cómo vivían los faraones? Lo descubriremos en...,www.civitatis.com/es/berlin/visita-guiada-muse...,www.civitatis.com/f/alemania/berlin/visita-gui...,NaN,[],[],2 horas,52.51956430488067,13.39765149325871,"Museumsinsel, Burgstraße, Spandauer Vorstadt, ...",7.50,EUR,Español
2,Free tour de los misterios y leyendas de Berlín,Los secretos de la capital alemana son numeros...,www.civitatis.com/es/berlin/free-tour-misterio...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/alemania/berlin/free-tour-...,[],[],3 horas,52.52062505416855,13.4071427198706,"St. Marienkirche, 8, Karl-Liebknecht-Straße, S...",2.20,EUR,Español
3,Tour de la Guerra Fría + Muro de Berlín + Muse...,Si queréis saber cómo era la vida en una ciuda...,www.civitatis.com/es/berlin/berlin-guerra-fria/,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/alemania/berlin/berlin-gue...,[],[],4 horas,52.52105,13.41017,"Gontardstraße, Mitte, Berlin, 10178, Deutschland",7.80,EUR,Español
4,Entrada al Icebar Berlín,Si queréis saber cómo se vive dentro de un igl...,www.civitatis.com/es/berlin/entrada-icebar-ber...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/alemania/berlin/entrada-ic...,[],[],45 minutos,52.521041,13.4040211,"2, Spandauer Straße, Spandauer Vorstadt, Mitte...",5.00,EUR,Entradas
5,Visita guiada por la Isla de los Museos,Acompañadnos en esta visita guiada por la Isla...,www.civitatis.com/es/berlin/visita-guiada-isla...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/alemania/berlin/visita-gui...,[],[],3 horas,52.517258,13.394698,"Unter den Linden, Friedrichswerder, Mitte, Ber...",4.00,EUR,Español
6,Tour en bicicleta por Berlín,En este tour en bicicleta por Berlín practicar...,www.civitatis.com/es/berlin/berlin-bicicleta/,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/alemania/berlin/berlin-bic...,[],[],3h 30m,52.54005526,13.41342434,"Kulturbrauerei, 36, Schönhauser Allee, Bremer ...",6.80,EUR,Inglés
7,"Autobús turístico de Berlín, City Sightseeing",El autobús turístico recorre Berlín realizando...,www.civitatis.com/es/berlin/autobus-turistico-...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/alemania/berlin/autobus-tu...,[],[],1 - 2d,0,0,NaN,9.60,EUR,Español
8,Entrada al Madame Tussauds de Berlín,Sacad a lucir vuestras mejores poses en el Mad...,www.civitatis.com/es/berlin/entrada-madame-tus...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/alemania/berlin/entrada-ma...,[],[],Entradas,52.5168789,13.3813841,NaN,5.80,EUR,NaN
9,Entrada a Panoramapunkt sin colas,Con esta entrada a Panoramapunkt subiréis en e...,www.civitatis.com/es/berlin/entrada-panoramapu...,"www.civitatis.comdata:image/gif;base64,R0lGODl...",www.civitatis.com/f/alemania/berlin/entrada-pa...,[],[],Entradas,52.509299280840885,13.375025669424874,"Jam Sound, 1, Potsdamer Straße, Tiergarten, Mi...",3.38,EUR,NaN


In [351]:
total_actitivities_df.shape

(5, 14)